In [115]:
# import dependancies
import pandas as pd
import random
from datetime import datetime, timedelta
from yahoo_fin.stock_info import *
import yfinance as yf
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from pprint import pprint

#alphavantage_api_key = 'AJP4EKR4LE52NNK4'
alphavantage_api_key = 'CORYDT1RMNVL3DVT'

In [78]:
file_path = 'nasdaq_screener_1700241077431.csv'

# Read the CSV file into a Pandas DataFrame
stock_list_df = pd.read_csv(file_path)

# Display the DataFrame
stock_list_df.head()

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
0,A,Agilent Technologies Inc. Common Stock,$112.505,-1.6850,-1.476%,3.291753e+10,United States,1999.0,777291,Industrials,Electrical Products
1,AA,Alcoa Corporation Common Stock,$26.42,0.4200,1.615%,4.715228e+09,United States,2016.0,1678614,Industrials,Aluminum
2,AAL,American Airlines Group Inc. Common Stock,$12.245,0.0550,0.451%,8.002616e+09,United States,NaN,9627454,Consumer Discretionary,Air Freight/Delivery Services
3,AAON,AAON Inc. Common Stock,$63.02,0.5300,0.848%,5.120051e+09,United States,NaN,87325,Industrials,Industrial Machinery/Components
4,AAP,Advance Auto Parts Inc.,$49.2001,-4.2199,-7.899%,2.925943e+09,United States,NaN,2518094,Consumer Discretionary,Auto & Home Supply Stores


In [79]:
# create an array of tickers

ticker_list = stock_list_df['Symbol'].values

In [98]:
# define function to generate random date, and ensure that the date is a weekday

def generate_random_date(start_date, end_date):
    while True:
        # Convert the date strings to datetime objects
        start_date = datetime.datetime.strptime(start_date, '%Y-%m-%d')
        end_date = datetime.datetime.now()  # Using today's date as the end date

        # Calculate the range of days between the start and end date
        delta_days = (end_date - start_date).days

        # Generate a random number of days within the range
        random_days = random.randint(0, delta_days)

        # Calculate the random date
        random_date = start_date + timedelta(days=random_days)

        # Check if the generated date is a weekday
        if random_date.weekday() < 5:
            return random_date



In [81]:
# define a function to choose a random stock from the list

def choose_random_stock(ticker_list):    
    
    # Randomly choose a stock from the list
    random_stock = random.choice(ticker_list)
    
    return random_stock
    

In [82]:
choose_random_stock(ticker_list)

'PGR'

In [83]:
# ________________________NEED TO FIX________________________________


# define a function to check if data exists for stock on the date 

def check_stock_date(ticker, date):
    # Check if the input date is a Friday
    if input_date.weekday() == 4:  # Friday
        # Set end date to the next Monday
        adj_end_date = input_date + timedelta(days=(7 - input_date.weekday()))
    else:
        # Set end date to the next day
        adj_end_date = input_date + timedelta(days=1)
    
    # using yahoo finance function to get stock data dataframe
    stock_data = get_data(ticker, start_date=input_date, end_date=adj_end_date)
    
    if not stock_data.empty:
        return True
    else:
        return False
    

In [84]:
# define a function to return stock industry

def return_stock_industry(symbol):
    # Check if the symbol is in the DataFrame
    if symbol in stock_list_df['Symbol'].values:
        # Retrieve the sector for the given symbol
        sector = stock_list_df.loc[stock_list_df['Symbol'] == symbol, 'Sector'].iloc[0]
        return sector
    else:
        return f"Symbol '{symbol}' not found in the stock list."

In [85]:

# define a function to get the adjusted closing price of a stock on a particular day

def get_stock_price(ticker, input_date):
    # Check if the input date is a Friday
    if input_date.weekday() == 4:  # Friday
        # Set end date to the next Monday
        adj_end_date = input_date + timedelta(days=(7 - input_date.weekday()))
    else:
        # Set end date to the next day
        adj_end_date = input_date + timedelta(days=1)
    
    # using yahoo finance function to get stock data dataframe
    stock_data = get_data(ticker, start_date=input_date, end_date=adj_end_date)
    # extracting the price 
    adj_close_price = stock_data['adjclose'].values[-1]
    
    return adj_close_price

In [86]:
"""
# loop to get weekly returns

for i in range(1, 5):
    data_dict['WR{i}']=(get_stock_price(ticker, date) - get_stock_price(ticker, date - timedelta(weeks = 1)))/get_stock_price(ticker, date - timedelta(weeks = 1))
    date = date - timedelta(weeks=1)
    
"""

"\n# loop to get weekly returns\n\nfor i in range(1, 5):\n    data_dict['WR{i}']=(get_stock_price(ticker, date) - get_stock_price(ticker, date - timedelta(weeks = 1)))/get_stock_price(ticker, date - timedelta(weeks = 1))\n    date = date - timedelta(weeks=1)\n    \n"

In [87]:
# define a function to get stock trading volume

def get_stock_volume(ticker, input_date):
    # Check if the input date is a Friday
    if input_date.weekday() == 4:  # Friday
        # Set end date to the next Monday
        adj_end_date = input_date + timedelta(days=(7 - input_date.weekday()))
    else:
        # Set end date to the next day
        adj_end_date = input_date + timedelta(days=1)
    
    # using yahoo finance function to get stock data dataframe
    stock_data = get_data(ticker, start_date=input_date, end_date=adj_end_date)
    
    #extracting the volume
    trading_vol = stock_data['volume'].values[-1]
    
    return trading_vol

In [88]:
# function to get 4 previous weekdays from given_date

def get_previous_weekdays(given_date):
    previous_weekdays = []
    previous_date = given_date - timedelta(days=1)
    while len(previous_weekdays) < 4:
        if previous_date.weekday() < 5:
            previous_weekdays.append(previous_date)
        previous_date -= timedelta(days=1)

    return previous_weekdays


In [89]:
"""# loop to get last 5 trading volumes
data_dict['vol1'] = get_stock_volume(ticker, date)

for i, weekday in enumerate(previous_weekdays):
    data_dict['vol{i+1}'] = get_stock_volume(ticker, weekday)
            
"""

"# loop to get last 5 trading volumes\ndata_dict['vol1'] = get_stock_volume(ticker, date)\n\nfor i, weekday in enumerate(previous_weekdays):\n    data_dict['vol{i+1}'] = get_stock_volume(ticker, weekday)\n            \n"

In [90]:
# function to get company's financial ratios by scraping stockanalysis.com

def get_stock_financials(ticker, given_date):
    
    financial_ratios = {}
    
    # URL of the website
    url = f"https://stockanalysis.com/stocks/{ticker}/financials/ratios/?p=quarterly"

    # Use Selenium to open the webpage and let JavaScript execute
    driver = webdriver.Chrome()  # Make sure you have the ChromeDriver executable in your PATH
    driver.get(url)

    # Allow some time for JavaScript to execute and load the content
    time.sleep(5)  # Adjust the sleep time based on your needs

    # Get the page source after JavaScript execution
    page_source = driver.page_source

    # Close the Selenium WebDriver
    driver.quit()

    # Parse the HTML content of the page
    soup = BeautifulSoup(page_source, "html.parser")

    # Find the table by class name
    table = soup.find("table", class_="w-full")

    # Check if the table is found
    if table:
        # Use pandas to read the HTML table into a DataFrame
        df = pd.read_html(str(table))[0]
        
    else:
        print("Table not found on the page.")
    
    # Extract date columns excluding the first column
    date_columns = df.columns[2:]

    # Convert date columns to datetime objects
    date_columns = pd.to_datetime(date_columns)

    # Find the closest date column to the given date that is earlier
    earlier_dates = [date for date in date_columns if date < given_date]
    closest_date_column = max(earlier_dates)

    # Access the closest column
    closest_column = closest_date_column.strftime('%Y-%m-%d')

    #extracting financial ratios
    
    financial_ratios['pe_ratio']=float(df.loc[df['Quarter Ended'] == 'PE Ratio', closest_column].values[0])
    financial_ratios['debt_to_equity']=float(df.loc[df['Quarter Ended'] == 'Debt / Equity Ratio', closest_column].values[0])
    financial_ratios['quick_ratio']=float(df.loc[df['Quarter Ended'] == 'Quick Ratio', closest_column].values[0])
    total_shareholder_return_str = df.loc[df['Quarter Ended'] == 'Total Shareholder Return', closest_column].values[0]
    financial_ratios['total_shareholder_return'] = float(total_shareholder_return_str.strip('%'))
    
    # URL of the website
    url = f"https://stockanalysis.com/stocks/{ticker}/financials/?p=quarterly"

    # Use Selenium to open the webpage and let JavaScript execute
    driver = webdriver.Chrome()  # Make sure you have the ChromeDriver executable in your PATH
    driver.get(url)

    # Allow some time for JavaScript to execute and load the content
    time.sleep(5)  # Adjust the sleep time based on your needs

    # Get the page source after JavaScript execution
    page_source = driver.page_source

    # Close the Selenium WebDriver
    driver.quit()

    # Parse the HTML content of the page
    soup = BeautifulSoup(page_source, "html.parser")

    # Find the table by class name
    table = soup.find("table", class_="w-full")

    # Check if the table is found
    if table:
        # Use pandas to read the HTML table into a DataFrame
        df = pd.read_html(str(table))[0]
        
    else:
        print("Table not found on the page.")
    
    profit_margin_str= df.loc[df['Quarter Ended'] == 'Profit Margin', closest_column].values[0]
    financial_ratios['profit_margin'] = float(profit_margin_str.strip('%'))
    cash_margin_str= df.loc[df['Quarter Ended'] == 'Free Cash Flow Margin', closest_column].values[0]
    financial_ratios['free_cash_margin'] = float(cash_margin_str.strip('%'))
    

    return financial_ratios
    
    

In [91]:
get_data('aapl', end_date = '01/01/2020')

,open,high,low,close,adjclose,volume,ticker
1980-12-12,0.128348,0.128906,0.128348,0.128348,0.099319,469033600,AAPL
1980-12-15,0.122210,0.122210,0.121652,0.121652,0.094137,175884800,AAPL
1980-12-16,0.113281,0.113281,0.112723,0.112723,0.087228,105728000,AAPL
1980-12-17,0.115513,0.116071,0.115513,0.115513,0.089387,86441600,AAPL
1980-12-18,0.118862,0.119420,0.118862,0.118862,0.091978,73449600,AAPL
...,...,...,...,...,...,...,...
2019-12-24,71.172501,71.222504,70.730003,71.067497,69.236237,48478800,AAPL
2019-12-26,71.205002,72.495003,71.175003,72.477501,70.609909,93121200,AAPL
2019-12-27,72.779999,73.492500,72.029999,72.449997,70.583115,146266000,AAPL
2019-12-30,72.364998,73.172501,71.305000,72.879997,71.002037,144114400,AAPL


In [92]:
def calculate_volatility(ticker, end_date):
    df=get_data(ticker, end_date=end_date)
    # Calculate daily returns
    df['daily_return'] = df['adjclose'].pct_change()

    # Calculate volatility as the standard deviation of daily returns
    volatility = df['daily_return'].std()

    return volatility

In [99]:
date= generate_random_date('2013-01-01', '2023-10-31')
ticker = choose_random_stock(ticker_list)
print(ticker, date)
calculate_volatility(ticker, date)

SYF 2019-03-04 00:00:00


0.01728126853941484

In [94]:
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=AAPL&time_from=20220101T0130&time_to=20200101T0130&sort=RELEVANCE&apikey={alphavantage_api_key}'
r = requests.get(url)
data = r.json()

pprint(data)

{'feed': [{'authors': ['Henry Khederian'],
           'banner_image': 'https://cdn.benzinga.com/files/images/story/2022/05/21/man-2425121.jpg?width=1200&height=800&fit=crop',
           'category_within_source': 'Trading',
           'overall_sentiment_label': 'Neutral',
           'overall_sentiment_score': 0.114801,
           'source': 'Benzinga',
           'source_domain': 'www.benzinga.com',
           'summary': 'Enphase Energy Inc ( NASDAQ: ENPH ) is known as a '
                      'global leader in solar energy solutions, and over the '
                      'past five years, delivered beaming returns for traders '
                      "and investors. Since May 2017, Enphase Energy stock's "
                      "5-year return has outperformed several of the world's "
                      'most popular cryptocurrencies,',
           'ticker_sentiment': [{'relevance_score': '0.977196',
                                 'ticker': 'MSFT',
                                 'ti

In [96]:

# Example datetime object
example_datetime = date

# Extract year, month, and day
year = example_datetime.year
month = example_datetime.month
day = example_datetime.day

# Display the results
print(f"Year: {year}")
print(f"Month: {month}")
print(f"Day: {day}")

start_date = date - timedelta(weeks=4)
print(start_date)

Year: 2023
Month: 4
Day: 15
2023-03-18 08:30:00


In [97]:
#------------------------NEED TO FIND A NEW SOURCE---------------------------------

def get_news_sentiment(ticker, end_date):
    
    # Extract year, month, and day
    end_date=date
    end_year = end_date.year
    end_month = str(end_date.month).zfill(2)
    end_day = str(end_date.day).zfill(2)
    
    start_date = end_date - timedelta(weeks=4)
    start_year = start_date.year
    start_month = str(start_date.month).zfill(2)
    start_day = str(start_date.day).zfill(2)
   

In [117]:
from datetime import datetime

timestamp_in_milliseconds = 1700197200000

# Convert milliseconds to seconds
timestamp_in_seconds = timestamp_in_milliseconds / 1000

# Convert timestamp to datetime
regular_time = datetime.utcfromtimestamp(timestamp_in_seconds).strftime('%Y-%m-%d %H:%M:%S')

print(regular_time)


2023-11-17 05:00:00
